# IMPORT THE LIBRARIES

In [23]:
from minio import Minio
import os
import requests
import requests.auth
import boto3
from botocore.client import Config
from botocore.exceptions import ClientError
import logging


## MAKE THE CONNECTION WITH MINIO

In [14]:
client = Minio('localhost:9000',
                    access_key='secret',
                    secret_key='secret',
              secure=False)

## LIST THE BUCKETS IN MINIO

In [15]:
buckets = client.list_buckets()

In [16]:
for bucket in buckets:
    print(bucket.name, bucket.creation_date)

chumtestbucket 2021-02-25 21:57:05.568000+00:00


# COMMAND TO CREATE BUCKET IN MINIO

In [17]:
client.make_bucket("chumtestbucket")

S3Error: S3 operation failed; code: BucketAlreadyOwnedByYou, message: Your previous request to create the named bucket succeeded and you already own it., resource: /chumtestbucket, request_id: 1684FC41DD894F76, host_id: c4798d9b-47b0-4e39-bbc4-82ec042e4593, bucket_name: chumtestbucket

# INITIALIZE BROTO CONNECTION WITH MINIO

In [18]:
s3 = boto3.resource('s3',aws_access_key_id = 'secret', aws_secret_access_key = 'secret', endpoint_url='http://recherche-coda19-rouge:9000',
                   config=Config(signature_version='s3v4'))

In [19]:
for bucket in s3.buckets.all():
    print(bucket.name)

chumtestbucket


# USE BROTO TO UPLOAD DATA TO A BUCKET IN MINIO

In [8]:
s3.Bucket('chumtestbucket').upload_file('culture_data.json.ndjson.gz','culture_data.json.ndjson.gz')

In [8]:
s3.Bucket('chumtestbucket').upload_file('episode_data.json.ndjson.gz','episode_data.json.ndjson.gz')

In [17]:
s3.Bucket('chumtestbucket').upload_file('demographic_data.json.ndjson.gz','demographic_data.json.ndjson.gz')

# USE BROTO TO DELETE DATA FROM A BUCKET IN MINIO

In [103]:
obj_to_delete = s3.Object('chumtestbucket', 'culture_data.ndjson.gz')
obj_to_delete.delete()

{'ResponseMetadata': {'RequestId': '16677711FB407564',
  'HostId': '',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'accept-ranges': 'bytes',
   'content-security-policy': 'block-all-mixed-content',
   'server': 'MinIO',
   'vary': 'Origin',
   'x-amz-request-id': '16677711FB407564',
   'x-xss-protection': '1; mode=block',
   'date': 'Sat, 27 Feb 2021 01:38:30 GMT'},
  'RetryAttempts': 0}}

# USE MINIO HANDLER TO UPLOAD DATA IN THE MINIO BUCKET (ANOTHER             

# APPROACH)

In [61]:
client.fput_object('chumtestbucket','culture_data.ndjson.gz','culture_data.ndjson.gz')    

# USE MINIO HANDLER TO REMOVE DATA FROM THE MINIO BUCKET 

# (ANOTHER  APPROACH)

In [90]:
client.remove_object('chumtestbucket','culture_data.ndjson.gz')

# METHOD TO CREATE URL POINTER TO A DATA LOCATED IN THE MINIO 

# BUCKET

In [22]:
def create_presigned_url(bucket_name, object_name, expiration=3600):
    """Generate a presigned URL to share an S3 object

    :param bucket_name: string
    :param object_name: string
    :param expiration: Time in seconds for the presigned URL to remain valid
    :return: Presigned URL as string. If error, returns None.
    """

    # Generate a presigned URL for the S3 object
    s3_client = boto3.client('s3', endpoint_url='http://recherche-coda19-rouge:9000',aws_access_key_id = 'secret', aws_secret_access_key = 'secret')
    
    try:
        response = s3_client.generate_presigned_url('get_object', Params={'Bucket': bucket_name,'Key': object_name}, ExpiresIn=expiration)
    
    except ClientError as e:
    
        logging.error(e)
        return None

    # The response contains the presigned URL
    return response

# CALL THE URL GENERATION FUNCTION

In [8]:
url = create_presigned_url('chumtestbucket', 'culture_data.json.ndjson.gz')

In [10]:
url = create_presigned_url('chumtestbucket', 'episode_data.json.ndjson.gz')

In [18]:
url = create_presigned_url('chumtestbucket', 'demographic_data.json.ndjson.gz')

# DISPLAY THE GENERATED URL

In [9]:
print(url)

http://recherche-coda19-orange:9000/chumtestbucket/culture_data.json.ndjson.gz?AWSAccessKeyId=_SNBlDc5gZf64xnLA0Sk_CJcTlmUQNRp&Signature=fpaujD0pU7ESW4Mr8VfB%2FG%2BFghA%3D&Expires=1622484351


# CREATE THE PAYLOAD

In [10]:
payload=({"source":url})

# GET THE USER AUTHORIZATION. THIS WAS DEFINED IN THE AIDBOX 

# FIRST

In [11]:
requests.get("http://recherche-coda19-orange:8888/User", auth=requests.auth.HTTPBasicAuth('python_load_script','chum123'))

<Response [200]>

# POST (UPLOAD) THE DATA ON THE AIDBOX

In [12]:
requests.post("http://recherche-coda19-orange:8888/$load",auth=requests.auth.HTTPBasicAuth('python_load_script','chum123'),json=payload)

<Response [200]>

# THE FOLLOWING LINES ARE ONLY FOR DEBUGGING. 

# ASSIGN user_content TO THE requests.get (LINE FROM ABOVE) 

# ASSIGN response.json AND response.text TO THE 

# requests.post LINE.

In [66]:
print(user_content.json())
print(response.json())
print(response.text())

{'resourceType': 'OperationOutcome', 'id': 'exception', 'text': {'status': 'generated', 'div': 'Cannot invoke "Object.toString()" because "s" is null'}, 'issue': [{'severity': 'fatal', 'code': 'exception', 'diagnostics': 'Cannot invoke "Object.toString()" because "s" is null'}]}
